In [241]:
import numpy as np
from random import *

In [242]:
learned = np.zeros([0,4])
obs = np.zeros([0,5])
epsilon = 0.1
for i in range(100):
    spielfeld = np.zeros([10,10])
    spielfeld[0,:] = 2
    spielfeld[9,:] = 2
    spielfeld[:,0] = 2
    spielfeld[:,9] = 2
    spielfeld[8,8] = 1
    spielfeld[4,2] = 1
    spielfeld[6,4] = 1
    spielfeld[3,7] = 1
    spielfeld[5,6] = 1
    spielfeld = spielfeld.astype(int)
    state = [1,1]
    learned, obs = do_action(state, spielfeld, learned, epsilon, obs)
# 1. Richtung des nächsten Coins: (1,2,3,4:links,oben,rechts,unten)
# 2.-5. Direkte Umgebung: (links, oben, rechts, unten; 0=frei, 1 = Coin, 2 = Wand)
# Actions: "down", "up", "left", "right", "wait"
# [3. 2. 2. 0. 0.]

---
Runden: 659
---
Runden: 825
---
---
---
---
---
---
---
Runden: 926
---
---
---
Runden: 819
---
Runden: 873
---
Runden: 450
---
---
---
---
---
---
---
---
Runden: 670
---
---
---
---
---
---
Runden: 526
---
---
---
---
---
---
Runden: 282
---
Runden: 574
---
---
Runden: 484
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
Runden: 758
---
---
---
Runden: 497
---
---
Runden: 542
---
---
---
Runden: 442
---
Runden: 393
---
Runden: 541
---
---
Runden: 280
---
---
---
---
Runden: 949
---
---
---
---
---
Runden: 550
---
---
---
Runden: 630
---
Runden: 302
---
---
---
---
Runden: 855
---
---
Runden: 247
---
---
---
---
---
Runden: 911
---
---
---
---
---
---
Runden: 888
---
---
---
---
---
---


In [237]:
def do_action(state, spielfeld, learned, epsilon, obs):
    actions = np.array(["down", "up", "left", "right"])
    discount = 1
    learning_rate = 1
    last_action_index = -1
    last_index = -1
    last_reward = -1
    coins_found = 0
    for i in range(1000):
        n_state = [0,0]
        if False:
            print("+++++++++++++++++++")
        else:
            indices = np.array([])
            possible_actions = np.array([])
            current_obs = get_observation(spielfeld,state[0],state[1])
            obs, index_current, learned = update_and_get_obs(obs, current_obs, learned)
            if epsilon > uniform(0,1):
                # epsilon greedy
                choice = np.random.choice(np.arange(actions.shape[0]))
            else:
                choice = np.random.choice(np.flatnonzero(learned[index_current] == learned[index_current].max()))
                
            my_best_value = np.max(learned[index_current])
            # print(actions[choice])
            n_state = get_following_state(state, spielfeld, actions[choice], actions, True)
            if not last_index == -1:
                learned[last_index, last_action_index] = (1-learning_rate) * learned[last_index, last_action_index] + learning_rate * (last_reward + discount * my_best_value)
            
            last_reward, spielfeld, coins_found = get_reward(state,spielfeld,actions[choice], i, coins_found)
            last_action_index = choice
            last_index = index_current
            state = n_state
            if coins_found == 5:
                print("Runden: " + str(i))
                break
    #print("learned:")
    #print(learned)
    #print("observations:")
    #print(obs)
    print("---")
    return learned, obs

In [240]:
def get_following_state(state, spielfeld, choice, actions, wall_sensitive):
    n_state = np.array([0,0])
    if(choice == "down"):
        n_state[0] = state[0] + 1
        n_state[1] = state[1]
    elif(choice == "up"):
        n_state[0] = state[0] - 1
        n_state[1] = state[1]
    elif(choice == "left"):
        n_state[0] = state[0]
        n_state[1] = state[1] - 1
    elif(choice == "right"):
        n_state[0] = state[0]
        n_state[1] = state[1] + 1
    elif(choice == "wait"):
        n_state[0] = state[0]
        n_state[1] = state[1]
        
    if wall_sensitive and spielfeld[n_state[0], n_state[1]] == 2:
        # return old state if action is not possible
        return state
    
    return n_state

def get_reward(state,spielfeld,action, step, coins_found):
    x,y = 0,0
    if action == "left":
        x = -1
    if action == "right":
        x = 1
    if action == "down":
        y = 1
    if action == "up":
        y = -1
    
    y +=state[0]
    x +=state[1] 
    
    copy = np.copy(spielfeld)
    copy[state[0], state[1]] = 8
    # print(copy)
    if spielfeld[y,x] == 1:
        # collect coin
        # print("COIN COLLECTED " + str(step))
        spielfeld[y,x] = 0
        return 1000, spielfeld, coins_found+1
    elif spielfeld[y,x] == 2:
        # Mauer
        return 0, spielfeld, coins_found
    else:
        return 0, spielfeld, coins_found

In [200]:
def get_observation(spielfeld, y, x):
    # Suche den Coin, der am nächsten ist (manhattan distance)
    coinsY, coinsX = np.where(spielfeld == 1)
    distancesY = np.abs(coinsY - y)
    distancesX = np.abs(coinsX - x)
    observation = np.array([0, 0,0,0,0]) 
    # 1. Richtung des nächsten Coins: (1,2,3,4:links,oben,rechts,unten)
    # 2. Direkte Umgebung: (links, oben, rechts, unten; 0=frei, 1 = Coin, 2 = Wand)
 
    if not distancesY.shape[0] == 0:
        distances = distancesY + distancesX
        coinY = coinsY[np.argmin(distances)]
        coinX = coinsX[np.argmin(distances)]
        # setze observation flags:
        if coinX < x:
            observation[0] = 1
        elif coinX > x:
            observation[0] = 3
        elif coinY > y:
            observation[0] = 2
        elif coinY < y:
            observation[0] = 4
    
    
    observation[1] = get_spielfeld(y, x - 1,spielfeld)
    observation[2] = get_spielfeld(y - 1, x,spielfeld)
    observation[3] = get_spielfeld(y, x + 1,spielfeld)
    observation[4] = get_spielfeld(y + 1, x,spielfeld)
    # links, oben, rechts, unten
    # print(x,y, observation)
    return observation

def update_and_get_obs(db, new_obs, learned):
    temp = -1
    for i in range(db.shape[0]):
        if np.array_equal(db[i], new_obs):
            return db, i, learned
    db = np.append(db,np.array([new_obs]), axis = 0)
    learned = np.append(learned, np.zeros([1,learned.shape[1]]), axis = 0)
    return db, (db.shape[0] - 1), learned

def get_spielfeld(y,x,spielfeld):
    if y > spielfeld.shape[0] - 1 or x > spielfeld.shape[1] - 1 or y < 0 or x < 0:
        #print("collision")
        return 2
    else:
        return spielfeld[y,x]